In [1]:
import os
import sys
import torch
import time

base_directory = "../"
sys.path.insert(0, base_directory)

from stable_diffusion2.latent_diffusion import LatentDiffusion
from stable_diffusion2.stable_diffusion import StableDiffusion
from stable_diffusion2.utils.model import *
from stable_diffusion2.utils.utils import SectionManager as section
from stable_diffusion2.utils.utils import *
from stable_diffusion2.model.clip.clip_embedder import CLIPTextEmbedder



from stable_diffusion2.model.unet.unet import UNetModel

from pathlib import Path

device = get_device()

INFO: Using CUDA device: NVIDIA GeForce RTX 3080 Ti


In [2]:
get_memory_status()

Free: 11086 MiB
Total: 12287 MiB
Used: 1201 MiB


(tensor(11086), tensor(12287))

In [3]:
latent_diffusion_model = LatentDiffusion(linear_start=0.00085,
            linear_end=0.0120,
            n_steps=1000,
            latent_scaling_factor=0.18215
            )

INFO: `device` is None. Falling back to current device.
INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.


In [4]:
get_memory_status()

Free: 11084 MiB
Total: 12287 MiB
Used: 1203 MiB


(tensor(11084), tensor(12287))

In [5]:
latent_diffusion_model.load_autoencoder()
# latent_diffusion_model.load_submodel_tree()

In [6]:
get_memory_status()

Free: 11084 MiB
Total: 12287 MiB
Used: 1203 MiB


(tensor(11084), tensor(12287))

In [7]:
latent_diffusion_model.first_stage_model.load_encoder()

In [8]:
get_memory_status()

Free: 10938 MiB
Total: 12287 MiB
Used: 1349 MiB


(tensor(10938), tensor(12287))

In [9]:
img = load_img("./scripts2/test_img.jpg").to(device)

In [10]:
encoded_img = latent_diffusion_model.autoencoder_encode(img)

In [13]:
get_memory_status()

Free: 5720 MiB
Total: 12287 MiB
Used: 6567 MiB


(tensor(5720), tensor(12287))

In [12]:
latent_diffusion_model.first_stage_model.unload_decoder()

In [15]:
get_memory_status()

Free: 5720 MiB
Total: 12287 MiB
Used: 6567 MiB


(tensor(5720), tensor(12287))

In [16]:
latent_diffusion_model.first_stage_model.load_decoder()

In [17]:
get_memory_status()

Free: 5592 MiB
Total: 12287 MiB
Used: 6695 MiB


(tensor(5592), tensor(12287))

In [18]:
decoded_img = latent_diffusion_model.autoencoder_decode(encoded_img)

OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 12.00 GiB total capacity; 10.95 GiB already allocated; 0 bytes free; 11.14 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [1]:
import os
import sys
import torch
import time

base_directory = "../"
sys.path.insert(0, base_directory)

from stable_diffusion2.latent_diffusion import LatentDiffusion
from stable_diffusion2.stable_diffusion import StableDiffusion
from stable_diffusion2.utils.model import *
from stable_diffusion2.utils.utils import SectionManager as section
from stable_diffusion2.utils.utils import *
from stable_diffusion2.model.clip.clip_embedder import CLIPTextEmbedder



from stable_diffusion2.model.unet.unet import UNetModel

from pathlib import Path

In [2]:
device = check_device()
encoder = initialize_encoder(device = device)
decoder = initialize_decoder(device = device)

INFO: `device` is None. Falling back to current device.
INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.
Starting section: encoder initialization...
INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.
Finished section: encoder initialization in 0.27 seconds

Starting section: decoder initialization...
INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.
Finished section: decoder initialization in 0.27 seconds



In [3]:
get_memory_status()

Free: 10730 MiB
Total: 12287 MiB
Used: 1557 MiB


(tensor(10730), tensor(12287))

In [4]:
autoencoder = load_autoencoder(device = device)

Starting section: autoencoder model loading, from c:\Users\igor-\.cloned\kcg-ml-sd1p4\input\model\autoencoder\autoencoder.ckpt...
INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.
Finished section: autoencoder model loading, from c:\Users\igor-\.cloned\kcg-ml-sd1p4\input\model\autoencoder\autoencoder.ckpt in 0.00 seconds

Starting section: casting autoencoder model to device and evaling...
Finished section: casting autoencoder model to device and evaling in 0.00 seconds



In [5]:
get_memory_status()

Free: 10730 MiB
Total: 12287 MiB
Used: 1557 MiB


(tensor(10730), tensor(12287))

In [6]:
autoencoder.encoder = encoder
autoencoder.decoder = decoder

In [8]:
img = load_img("./scripts2/test_img.jpg").to(device)
encoded_img = autoencoder.encode(img)

In [9]:
get_memory_status()

Free: 1199 MiB
Total: 12287 MiB
Used: 11088 MiB


(tensor(1199), tensor(12287))

In [12]:
sample = encoded_img.sample()

In [13]:
sample.shape

torch.Size([1, 4, 64, 64])

In [14]:
torch.cuda.empty_cache()

In [15]:
get_memory_status()

Free: 1260 MiB
Total: 12287 MiB
Used: 11027 MiB


(tensor(1260), tensor(12287))

In [11]:
with section("autoencoder initialization from saved submodels"):
    autoencoder_3 = Autoencoder(emb_channels=4, z_channels=4)
    autoencoder_3.load_submodels(encoder_path=ENCODER_PATH, decoder_path=DECODER_PATH)

Starting autoencoder initialization from saved submodels...
Finished autoencoder initialization from saved submodels in 0.25 seconds


In [12]:
encoded_img_3 = autoencoder_3.encoder(img)

In [13]:
encoded_img.shape, encoded_img_3.shape

(torch.Size([1, 8, 64, 64]), torch.Size([1, 8, 64, 64]))

In [14]:
torch.norm(encoded_img_3 - encoded_img)

tensor(0., device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)

In [ ]:
latent_diffusion_model = 